In [1]:
import dotenv
import nest_asyncio

dotenv.load_dotenv()
nest_asyncio.apply()

In [2]:
import os
from atproto import Client

client = Client()
client.login(os.environ["BSKY_USR"], os.environ["BSKY_PWD"])

/home/cole/school/Trust-and-Safety/HW3/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'default' attribute with value None was provided to the `Field()` function, which has no effect in the context it was used. 'default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


ProfileViewDetailed(did='did:plc:y3nd3tn2labpv3ykyzu6c44p', handle='alexiosfanclub.bsky.social', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:y3nd3tn2labpv3ykyzu6c44p/bafkreig2s7qacccfw3kvnqnbv7dsaymv3r7zaguvvzz5vakfxgolue5xdy@jpeg', banner=None, created_at='2025-10-30T17:41:54.451Z', description=None, display_name='', followers_count=0, follows_count=1, indexed_at='2025-10-30T17:41:54.451Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=0, pronouns=None, status=None, verification=None, viewer=ViewerState(activity_subscription=None, blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=Fal

## List Trending Topics

In [3]:
topic_response = client.app.bsky.unspecced.get_trending_topics()
topics = topic_response.topics

for topic in topics:
    print(topic.topic)
    print("-----")

NYC Mayoral Election
-----
ACA
-----
Jonathan Greenblatt
-----
UPS Crash
-----
Mamdani
-----
Dodgers
-----
World Series
-----
Blue Jays
-----


## Get posts from a specific feed

In [4]:
from atproto_client.models.app.bsky.feed.defs import FeedViewPost
from atproto_client.models.app.bsky.unspecced.defs import TrendingTopic
from atproto_client.models.app.bsky.feed.get_feed import Params as GetFeedParams

def _format_feed_link(feed: str | TrendingTopic) -> str:
    """Format the feed link for a given trending topic.
    Args:
        topic (TrendingTopic): The trending topic object.
    Returns:
        str: The formatted feed link.
    """
    feed = feed.link if isinstance(feed, TrendingTopic) else feed

    user_handle = feed.split('/')[4]
    feed_id = feed.split('/')[-1]
    user = client.get_profile(user_handle)
    feed_link = f"at://{user.did}/app.bsky.feed.generator/{feed_id}"
    return feed_link

def get_feed_posts(feed: str | TrendingTopic, num_posts: int = 100) -> list[FeedViewPost]:
    """Get posts from a specific feed based on a trending topic.
    Args:
        topic (TrendingTopic): The trending topic object.
    Returns:
        list[FeedViewPost]: A list of posts from the feed.
    """
    feed_link = _format_feed_link(feed)

    if num_posts > 100:
        # Iteratively fetch posts in chunks of 100
        all_posts = []
        cursor = None
        while len(all_posts) < num_posts:
            limit = min(100, num_posts - len(all_posts))
            posts = client.app.bsky.feed.get_feed(GetFeedParams(feed=feed_link, limit=limit, cursor=cursor))

            all_posts.extend(posts.feed)
            cursor = posts.cursor
            if not cursor:
                return all_posts 
        return all_posts
    else:
        posts = client.app.bsky.feed.get_feed(GetFeedParams(feed=feed_link, limit=num_posts))

    return posts.feed

In [8]:
target_feed = "https://bsky.app/profile/trending.bsky.app/feed/478223028" # Copied from trending topics on browser

feed = get_feed_posts(target_feed, 200)

In [11]:
feed[0].post.record

Record(created_at='2025-11-05T20:18:31.649Z', text='George Santos says he’s leaving New York City after last night’s election.\n\nZohran Mamdani is already making the city a better place.', embed=Main(images=[Image(alt='', image=BlobRef(mime_type='image/jpeg', size=201514, ref=IpldLink(link='bafkreie6qj2g36iakzp2vp2ev33vmgkqmxymktp37aps5crkcxp5wuk75m'), py_type='blob'), aspect_ratio=AspectRatio(height=585, width=430, py_type='app.bsky.embed.defs#aspectRatio'), py_type='app.bsky.embed.images#image')], py_type='app.bsky.embed.images'), entities=None, facets=None, labels=None, langs=['en'], reply=None, tags=None, py_type='app.bsky.feed.post')

## Save results

In [ ]:
import os
import json

exclude_fields = {
    'cid', 'post', 'author', 'embed', 'replyCount', 'repostCount', 'likeCount', 
    'indexed_at', 'langs', 'py_type','uri', 'threadgate', 'viewer'
}

def save_feed(feed: str | TrendingTopic, num_posts: int = 100):
    """Saves posts from a feed link to a json file in "saved_feeds" folder
    
    Args:
        feed (str | TrendingTopic): The feed link or TrendingTopic object
        num_posts (int): The number of posts to save

    """
    feed = feed.link if isinstance(feed, TrendingTopic) else feed

    posts = get_feed_posts(feed, num_posts)

    to_export = [post.post.model_dump(exclude=exclude_fields) for post in posts]

    with open(os.path.join("saved_feeds", f"{feed.split('/')[-1]}.json"), "w") as f:
        json.dump(to_export, f)

In [13]:
feed_to_save = "https://bsky.app/profile/trending.bsky.app/feed/478223028"
save_feed(feed=feed_to_save, num_posts=300)